# Biblioteca Financeira INCO

> – Dr. Bob Niedorf: How old is a person born in 1928?<br/>
– George Malley: Man or a woman?<br/>
– Dr. Bob Niedorf: Why?<br/>
– George Malley: Specifics, Bob.<br/>
– Dr. Bob Niedorf: Okay, one more time. How old is a MAN born in 1928?<br/>
– George Malley: Still alive?<br/>
– Dr. Bob Niedorf:  If a man is born in 1928, and he’s still alive, how old is he?<br/>
– George Malley: What month?<br/>
– Dr. Bob Niedorf: If a man was born October 3rd, 1928, and he’s still alive, how old is he?<br/>
– George Malley: What time?<br/>
– Dr. Bob Niedorf: 10 o’clock… PM!<br/>
– George Malley: Where?<br/>
– Dr. Bob Niedorf: Anywhere!<br/>
– George Malley: Well, let’s get specific, Bob! I mean, if the guy’s still alive, born in California, October 3rd, 1928, 10 PM, he’s 67 years, 9 months, 22 days, 14 hours, and…[takes Bob’s hand to see his wristwatch]… and 12 minutes. If he was born in New York, he’s 3 hours older, now isn’t he?<br/>
>
> — John Travolta, [Phenomenom](https://www.imdb.com/title/tt0117333).

Este caderno documenta a Biblioteca Financeira da INCO. Seu objetivo é definir com a maior especificidade possível o funcionamento de suas rotinas. Elas vão suportar uma combinação de operações – Bullet, Price, Juros mensais, e Livre – com indexadores – Pré-fixado, CDI, Poupança e IPCA. Mas com a ressalva de que Price não comporta indexadores flutuantes (CDI, Poupança e IPCA) dado que seus pagamentos devem ser fixos.

Observe que Livre é terminologia interna INCO, não é denominação usual de mercado. Foi criada para permitir operações de crédito com um nível máximo de flexibilidade.

Observe também que historicamente tivemos uma modalidade de operação que foi posteriormente abandonada, a Pool. Nossa base de código contém vestígios dela, especialmente nas tradutoras. Mas não são ativamente mantidas.

Além disso, os pagamentos gerados pelo motor financeiro deste caderno podem apresentar prestações de antecipação, inseridas na montagem do esqueleto do cronograma de acordo com a data em que ocorreu o evento de antecipação.

##  Conceitos

1. Amortização  
> Amortização é um processo de extinção de uma dívida através de pagamentos periódicos, que são realizados em função de um planejamento, de modo que cada prestação corresponde à soma do reembolso do capital ou do pagamento dos juros do saldo devedor, podendo ser o reembolso de ambos, sendo que os juros são sempre calculados sobre o saldo devedor.
>
> — [Amortização](https://pt.wikipedia.org/wiki/Amortização), Wikipédia.

2. Antecipação
> A antecipação é caracterizada pela amortização do principal fora do cronograma ordinário da modalidade de crédito e pode assumir caráter parcial ou total. A amortização de uma parte da dívida restante é chamada de parcial, enquanto a amortização que engloba toda essa dívida é chamada de total.

In [41]:
# Installing non-native libraries.
!python3 -m pip install typeguard
!python3 -m pip install git+https://github.com/inco-org/fincore.git
!python3 -m pip install ipywidgets==7.7.2  # FIXME: Higher versions are not working.

  Cloning https://github.com/inco-org/fincore.git to /tmp/pip-req-build-g9m2j0bx
  Running command git clone --filter=blob:none --quiet https://github.com/inco-org/fincore.git /tmp/pip-req-build-g9m2j0bx
  Resolved https://github.com/inco-org/fincore.git to commit a8bdd0fbd5532861dad132dbe6429023d5d13681
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Bullet

Também conhecido como Sistema de Amortização Americano Bullet – SAA Bullet.

Nesta modalidade o principal é devolvido em uma única parcela no final do prazo de  financiamento, e os juros de cada período são incorporados ao capital   (refinanciado) e também liquidados de uma única vez juntamente com o principal. Observe que o Sistema Americano pode ser reduzido a um Fluxo de Caixa com uma entrada e apenas uma amortização, com prazo de carência igual ao prazo do financiamento.

### Indexador Pré-fixado

Na Inco usa-se base diária considerando-se ano de 360 ou 365 dias. No entanto, a base de 365 foi abandonada pela Inco e só é suportada nesse motor para atender as operações antigas.

Cálculo do juros.

\begin{align}
FJ &= (1 + \frac{APY}{100})^\frac{dp}{B} \\
J &= (FJ - 1) \times P
\end{align}

Cálculo da prestação.

\begin{align}
PMT &= P + J
\end{align}

Em que

* $FJ$ é o fator de juros; e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $dp$ é o número de dias corridos entre a data do início do rendimento e a data de cálculo; e
* $B$ é a base de cálculo diária, ou o número de dias corridos em 1 (um) ano; e
* $J$ é o valor do juros; e
* $P$ é o valor do principal; e
* $PMT$ é o valor da prestação.

### Indexador CDI

Na Inco usa-se base diária considerando-se ano de 252 dias.

Cálculo do juros.

\begin{align}
FJ &= (1 + \frac{APY}{100})^\frac{dp}{252} \\
FDI &= \prod_{k=1}^n (1 + (PI \times \frac{TDI_k}{100})) \\
J &= (FJ \times FDI - 1) \times P
\end{align}

Cálculo da prestação.

\begin{align}
PMT &= P + J
\end{align}

Em que

* $FJ$ é o fator de juros (ou Spread); e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $dp$ é o número de dias úteis entre a data do início do rendimento e a data de cálculo; e
* $FDI$ é o fator de DI; e
* $PI$ é o percentual do índice a ser considerado; e
* $TDI$ é a taxa diária percentual do CDI, divulgada pelo BACEN; e
* $n$ é o número de taxas DI divulgadas no período de rendimento; e
* $k$ é o número de ordem da taxa DI; e
* $J$ é o valor do juros; e
* $P$ é o valor do principal; e
* $PMT$ é o valor da prestação.

### Indexador Poupança

Na Inco usa-se base diária considerando-se ano de 360 dias.

Cálculo do juros.

\begin{align}
FJ &= (1 + \frac{APY}{100})^\frac{dp}{360} \\
FP &= \prod_{k=1}^n (1 + (PI \times  \frac{TP_k}{100})) \\
J &= (FJ \times FP - 1) \times P
\end{align}

Cálculo da prestação.

\begin{align}
PMT &= P + J
\end{align}

Em que

* $FJ$ é o fator de juros (ou Spread); e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $dp$ é o número de dias corridos entre a data do início do rendimento e a data de cálculo; e
* $FP$ é o fator de Poupança; e
* $PI$ é o percentual do índice a ser considerado; e
* $TP$ é a taxa mensal percentual da Poupança, divulgada pelo BACEN; e
* $n$ é o número de taxas divulgadas no período de rendimento, sendo consideradas só aquelas cujos dias de início e término sejam iguais ao dia do aniversário (ou liquidação) da operação; e
* $k$ é o número de ordem da taxa da Poupança; e
* $J$ é o valor do juros; e
* $P$ é o valor do principal; e
* $PMT$ é o valor da prestação.

### Indexador ICPA

Na Inco usa-se base diária considerando-se ano de 360 dias.

Cálculo da correção.

\begin{align}
FC &= \prod_{k=1}^n (1 + \frac{VM_k}{100})^\frac{dp_k}{dt_k} \\
C &= (FC - 1) \times P
\end{align}

Cálculo do juros.

\begin{align}
FJ &= (1 + \frac{APY}{100})^\frac{dp}{360} \\
J &= (FJ - 1) \times (P + C)
\end{align}

Cálculo da prestação.

\begin{align}
PMT &= P + C + J
\end{align}

Em que

* $FC$ é o fator de correção do IPCA; e
* $VM$ é a variação mensal percentual do IPCA, divulgada pelo BACEN; e
* $dp_k$ é o número de dias corridos entre a data de início da correção e a data do cálculo, ou a data de fim da correção, caso seja menor; e
* $dt_k$ é o número de dias corridos entre a data de início da correção e a data de fim da correção; e
* $n$ é o número de taxas de correção divulgadas no período de rendimento; e
* $k$ é o número de ordem da taxa de correção; e
* $C$ é o valor da correção; e
* $P$ é o valor do principal; e
* $FJ$ é o fator de juros; e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $dp$ é o número de dias corridos entre a data do início do rendimento e a data de cálculo; e
* $J$ é o valor do juros; e
* $PMT$ é o valor da prestação.

## Juros mensais

Também conhecido como Sistema de Amortização Americano Padrão – SAA Pardão.

Nesta modalidade, o principal é também devolvido em uma única parcela no vencimento e os juros são pagos no final de cada período.

### Indexador Pré-fixado

Na Inco usa-se base diária considerando-se mês de 30 dias e ano de 360 dias.

Cálculo do juros.

\begin{align}
FJ &= [(1 + \frac{APY}{100})^\frac{30}{360}]^\frac{dp}{dt} \\
J &= (FJ - 1) \times P
\end{align}

Cálculo da prestação.

\begin{align}
PMT &=
\left\{
\begin{array}{cl}
    J & i \ne m \\
    P + J & i = m.
\end{array}
\right.
\end{align}

Em que

* $FJ$ é o fator de juros; e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $dp$ é o número de dias corridos entre a data do início do rendimento e a data de cálculo; e
* $dt$ é o número de dias corridos entre a data do início do rendimento e a data do fim do rendimento; e
* $J$ é o valor do juros; e
* $P$ é o valor do principal; e
* $PMT$ é o valor da prestação; e
* $i$ é o número da prestação; e
* $m$ é o prazo do rendimento, em meses.

### Indexador CDI

Na Inco usa-se base diária considerando-se ano de 252 dias.

Cálculo do juros.

\begin{align}
FJ &= (1 + \frac{APY}{100})^\frac{dp}{252} \\
FDI &= \prod_{k=1}^n (1 + (PI \times \frac{TDI_k}{100})) \\
J &= (FJ \times FDI - 1) \times P
\end{align}

Cálculo da prestação.

\begin{align}
PMT &=
\left\{
\begin{array}{cl}
    J & i \ne m \\
    P + J & i = m.
\end{array}
\right.
\end{align}

Em que

* $FJ$ é o fator de juros (ou Spread); e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $dp$ é o número de dias úteis entre a data do início do rendimento e a data de cálculo; e
* $FDI$ é o fator de DI; e
* $PI$ é o percentual do índice a ser considerado; e
* $TDI$ é a taxa diária percentual do CDI, divulgada pelo BACEN; e
* $n$ é o número de taxas DI divulgadas no período de rendimento; e
* $k$ é o número de ordem da taxa DI; e
* $J$ é o valor do juros; e
* $P$ é o valor do principal; e
* $PMT$ é o valor da prestação; e
* $i$ é o número da prestação; e
* $m$ é o prazo do rendimento, em meses.

### Indexador ICPA

Na Inco usa-se base diária considerando-se mês de 30 dias e ano de 360 dias.

Cálculo da correção.

\begin{align}
FC &= \prod_{k=1}^n (1 + \frac{VM_k}{100})^\frac{dp_k}{dt_k} \\
C &= (FC - 1) \times P
\end{align}

Cálculo do juros.

\begin{align}
FJ &= [(1 + \frac{APY}{100})^\frac{30}{360}]^\frac{dp}{dt} \\
J &= (FS - 1) \times (P + C)
\end{align}

Cálculo da prestação.

\begin{align}
PMT &=
\left\{
\begin{array}{cl}
    J & i \ne m \\
    P + C + J & i = m.
\end{array}
\right.
\end{align}

Em que

* $FC$ é o fator de correção do IPCA; e
* $VM$ é a variação mensal percentual do IPCA, divulgada pelo BACEN; e
* $C$ é o valor da correção; e
* $dp_k$ é o número de dias corridos entre a data de início da correção e a data do cálculo, ou a data de fim da correção, caso seja menor; e
* $dt_k$ é o número de dias corridos entre a data de início da correção e a data de fim da correção; e
* $n$ é o número de taxas de correção divulgadas entre um (1) mês anterior à data de início da primeira prestação e um (1) mês anterior à data de início da prestação calculada; e
* $k$ é o número de ordem da taxa de correção; e
* $P$ é o valor do principal; e
* $FJ$ é o fator de juros; e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $dp$ é o número de dias corridos entre a data do início do rendimento e a data de cálculo; e
* $dt$ é o número de dias corridos entre a data do início do rendimento e a data do fim do rendimento; e
* $J$ é o valor do juros; e
* $PMT$ é o valor da prestação; e
* $i$ é o número da prestação; e
* $m$ é o prazo do rendimento, em meses.


## Price

Tabela Price, TP, ou Sistema Francês de Amortização – SFA.

> “A denominação do Sistema de Amortização Francês vem do fato de ter sido utilizado primeiramente na França, no século XIX.”
>
> — SAMANEZ, Carlos Patricio. Matemática Financeira: Aplicações à análise de investimentos. São Paulo, Pearson Education do Brasil, ed 3, 2002.

Na Tabela PRICE, todas as prestações são iguais, periódicas e sucessivas.

### Indexador Pré-fixado

Na Inco usa-se base diária considerando-se mês de 30 dias e ano de 360 dias.

Cálculo da prestação.

\begin{align}
TM &= (1 + \frac{APY}{100})^\frac{30}{360} - 1 \\
PMT &= \frac{P \times TM}{1 - \frac{1}{(1 + TM)^n}}
\end{align}

Cálculo do juros.

\begin{align}
FJ &= [(1 + \frac{APY}{100})^\frac{30}{360}]^\frac{dp}{dt} \\
J &= (FJ - 1) \times BAL
\end{align}

Cálculo da amortização.

\begin{align}
A &= PMT - J_f
\end{align}

Em que

* $TM$ é a taxa mensal de juros; e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $PMT$ é o valor da prestação; e
* $P$ é o valor do principal; e
* $n$ é o prazo do rendimento, em meses.
* $FJ$ é o fator de juros; e
* $dp$ é o número de dias corridos entre a data do início do rendimento e a data de cálculo; e
* $dt$ é o número de dias corridos entre a data do início do rendimento e a data do fim do rendimento; e
* $J$ é o valor do juros; e
* $BAL$ é o valor do saldo devedor (Balance).
* $A$ é o valor da amortização do principal; e
* $J_f$ é o valor final do juros, calculado na data de fim do rendimento.

## Livre

Nesta modalidade o pagamento do principal e do juros é customizável. Dessa forma, é possível determinar o percentual de amortização e se haverá, ou não, pagamento do juros acumulado em cada prestação.

### Indexador Pré-fixado

Na Inco usa-se base diária considerando-se mês de 30 dias e ano de 360 dias.

Cálculo da amortização.

\begin{align}
A &= P \times \frac{PCT}{100}
\end{align}

Cálculo de juros.

\begin{align}
FJ &= [(1 + \frac{APY}{100})^\frac{30}{360}]^\frac{dp}{dt} \\
J &= (FJ - 1) \times BAL
\end{align}

Cálculo da prestação.

\begin{align}
PMT &= A + J
\end{align}

Em que

* $A$ é o valor da amortização do principal; e
* $P$ é o valor do principal; e
* $PCT$ é o percentual de amortização; e
* $FJ$ é o fator de juros; e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $dp$ é o número de dias corridos entre a data do início do rendimento e a data de cálculo; e
* $dt$ é o número de dias corridos entre a data do início do rendimento e a data do fim do rendimento; e
* $J$ é o valor do juros; e
* $BAL$ é o valor do saldo devedor (Balance) anterior à amortização; e
* $PMT$ é o valor da prestação.

### Indexador CDI

Na Inco usa-se base diária considerando-se ano de 252 dias.

Cálculo da amortização.

\begin{align}
A &= P \times \frac{PCT}{100}
\end{align}

Cálculo do juros.

\begin{align}
FJ &= (1 + \frac{APY}{100})^\frac{dp}{252} \\
FDI &= \prod_{k=1}^n (1 + (PI \times \frac{TDI_k}{100})) \\
J &= (FJ \times FDI - 1) \times P
\end{align}

Cálculo da prestação.

\begin{align}
PMT &= A + J
\end{align}

Em que

* $A$ é o valor da amortização do principal; e
* $P$ é o valor do principal; e
* $PCT$ é o percentual de amortização; e
* $FJ$ é o fator de juros; e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $dp$ é o número de dias úteis entre a data do início do rendimento e a data de cálculo; e
* $FDI$ é o fator de DI; e
* $PI$ é o percentual do índice a ser considerado; e
* $TDI$ é a taxa diária percentual do CDI, divulgada pelo BACEN; e
* $n$ é o número de taxas DI divulgadas no período de rendimento; e
* $k$ é o número de ordem da taxa DI; e
* $J$ é o valor do juros; e
* $B$ é o valor do saldo devedor (Balance) anterior à amortização; e
* $PMT$ é o valor da prestação.

### Indexador IPCA

Na Inco usa-se base diária considerando-se mês de 30 dias e ano de 360 dias.

Cálculo da amortização.

\begin{align}
A &= P \times \frac{PCT}{100}
\end{align}

Cálculo da correção.

\begin{align}
FC &= \prod_{k=1}^n (1 + \frac{VM_k}{100})^\frac{dp_k}{dt_k} \\
C &= (FC - 1) \times A
\end{align}

Cálculo do juros.

\begin{align}
FJ &= [(1 + \frac{APY}{100})^\frac{30}{360}]^\frac{dp}{dt} \\
J &= (FS - 1) \times (BAL + C)
\end{align}

Cálculo da prestação.

\begin{align}
PMT &= A + C + J
\end{align}
Em que

* $A$ é o valor da amortização do principal; e
* $P$ é o valor do principal; e
* $PCT$ é o percentual de amortização; e
* $FC$ é o fator de correção do IPCA; e
* $VM$ é a variação mensal percentual do IPCA, divulgada pelo BACEN; e
* $C$ é o valor da correção; e
* $BAL$ é o valor do saldo devedor (Balance) anterior à amortização; e
* $dp_k$ é o número de dias corridos entre a data de início da correção e a data do cálculo, ou a data de fim da correção, caso seja menor; e
* $dt_k$ é o número de dias corridos entre a data de início da correção e a data de fim da correção; e
* $n$ é o número de taxas de correção divulgadas entre um (1) mês anterior à data de início da primeira prestação e um (1) mês anterior à data de início da prestação calculada; e
* $k$ é o número de ordem da taxa de correção; e
* $FJ$ é o fator de juros; e
* $APY$ é a taxa anual percentual (Annual Percentage Yield); e
* $dp$ é o número de dias corridos entre a data do início do rendimento e a data de cálculo; e
* $dt$ é o número de dias corridos entre a data do início do rendimento e a data do fim do rendimento; e
* $J$ é o valor do juros; e
* $PMT$ é o valor da prestação.

## Observações

### Sobre o fator de correção para o IPCA

As taxas de variação mensal ($VM$) utilizadas para calcular o fator de correção ($FC$) são determinadas por uma data base ($D$), o período ($p$) a ser considerado, em meses, e um deslocamento ($s$), também mensal, de modo que:

\begin{align}
s &=
\left\{
\begin{array}{cl}
    0 & AUTO \\
    1 \: mês & M-1 \\
    2 \: meses & M-2.
\end{array}
\right.
\end{align}

\begin{align}
D - s \le d_{VM} \le D + p - s
\end{align}

Onde $d_{VM}$ é a data da taxa de variação analisada.

### Sobre a antecipação parcial para modalidades com amortizações periódicas

A inserção de uma antecipação parcial na montagem do cronograma para alguma das modalidades de crédito que possuem amortizações periódicas no fluxo de pagamentos ordinário (Price ou Livre) faz com que os percentuais de amortização do principal posteriores à essa antecipação precisem ser atualizados. Essa atualização é feita de forma que o novo percentual de amortização ($P_{novo}$), de uma prestação arbitrária, deve ser igual ao percentual antigo ($P_{antigo}$) multiplicado por um fator de ajuste ($F$):

\begin{align}
F &= \frac{A_{corrente}}{A_{ordinária}} \\
P_{novo} &= P_{antigo} * F
\end{align}

Onde $A_{corrente}$ é o percentual de amortização restante do fluxo de pagamentos corrente, em cima do principal e incluindo as amortizações extraordinárias (antecipações), e $A_{ordinário}$ é o percentual de amortização restante do fluxo de pagamentos ordinário.

# Kichen Sink

In [68]:
import decimal
import fincore
import datetime

_AMORT = lambda x: fincore.Amortization(date=datetime.date.fromisoformat((y := x.split(','))[0]), amortization_ratio=decimal.Decimal(y[1]), amortizes_interest=y[2] == 'true')

_PLA_AMORT = lambda x: fincore.Amortization(date=datetime.date.fromisoformat(y[0]), amortization_ratio=decimal.Decimal(y[1]), amortizes_interest=y[2] == 'true', price_level_adjustment=fincore.PriceLevelAdjustment('IPCA', base_date=datetime.date.fromisoformat(y[4]), period=int(y[5]), shift=y[6], amortizes_adjustment=y[6] == 'true')) if len(y := x.split(',')) > 3 and y[3] == 'IPCA' else _AMORT(x)

OPS = {
    'Villa VIC Pisa - Pré-Fixada - Bullet': list(fincore.build_bullet(
        principal=decimal.Decimal('1609500'),
        apy=decimal.Decimal('18.5'),
        zero_date=datetime.date(2022, 3, 9),
        term=12,
        capitalisation='365'
    )),

    'Villa VIC Pisa - Pré-Fixada - Juros mensais': list(fincore.build_jm(
        principal=decimal.Decimal('890500'),
        apy=decimal.Decimal('18.5'),
        zero_date=datetime.date(2022, 3, 9),
        term=12
    )),

    'Villa VIC Pisa - CDI - Bullet': list(fincore.build_bullet(
        principal=decimal.Decimal('1944500'),
        apy=decimal.Decimal('6.0'),
        zero_date=datetime.date(2022, 3, 9),
        term=12,
        vir=fincore.VariableIndex('CDI')
    )),

    'Villa VIC Pisa - CDI - Juros mensais': list(fincore.build_jm(
        principal=decimal.Decimal('555500'),
        apy=decimal.Decimal('6.0'),
        zero_date=datetime.date(2022, 3, 9),
        term=12,
        vir=fincore.VariableIndex('CDI')
    )),

    'Credlar Chopin - Pré-Fixada - Price': list(fincore.build_price(
        principal=decimal.Decimal('481000'),
        apy=decimal.Decimal('19.0'),
        zero_date=datetime.date(2022, 3, 4),
        term=24
    )),

    'Residencial Arnaldo Patrus 2 - Pré-Fixada - Price': list(fincore.build_price(
        principal=decimal.Decimal('151000'),
        apy=decimal.Decimal('18.0'),
        zero_date=datetime.date(2022, 3, 4),
        term=24
    )),

    'Felicidade Residencial Clube - Pré-Fixada - Bullet': list(fincore.build_bullet(
        principal=decimal.Decimal('978000'),
        apy=decimal.Decimal('13.5'),
        zero_date=datetime.date(2021, 7, 23),
        term=9,
        capitalisation='365'
    )),

    'Felicidade Residencial Clube - Pré-Fixada - Juros mensais': list(fincore.build_jm(
        principal=decimal.Decimal('222000'),
        apy=decimal.Decimal('13.5'),
        zero_date=datetime.date(2021, 7, 23),
        term=9
    )),

    'YUCA SPE V - IPCA - Juros mensais': list(fincore.build_jm(
        principal=decimal.Decimal('1844500'),
        apy=decimal.Decimal('7.0'),
        zero_date=datetime.date(2022, 2, 18),
        vir=fincore.VariableIndex('IPCA'),
        term=30
    )),

    # Skeleton:
    #
    #  • principal fincore.Amortization;
    #  • interest fincore.Amortization;
    #  • price leveling;
    #  • price leveling base date;
    #  • price leveling period;
    #  • price leveling shift mode (M-1, M-2, AUTO);
    #  • price level fincore.Amortization.
    #
    'ASAD Energia - IPCA - Livre': list(fincore.build(
        principal=decimal.Decimal('145000'),
        apy=decimal.Decimal('10.0'),
        vir=fincore.VariableIndex('IPCA'),
        amortizations=list(map(_PLA_AMORT, '''
            2022-04-05,0,false
            2022-05-05,0.0130614411,true
            2022-06-05,0.0131655949,true
            2022-07-05,0.0132705792,true
            2022-08-05,0.0133764006,true
            2022-09-05,0.0134830659,true,IPCA,2021-07-01,12,AUTO,true
            2022-10-05,0.0135905818,true,IPCA,2021-07-01,12,AUTO,true
            2022-11-05,0.0136989550,true,IPCA,2021-07-01,12,AUTO,true
            2022-12-05,0.0138081924,true,IPCA,2021-07-01,12,AUTO,true
            2023-01-05,0.0139183008,true,IPCA,2021-07-01,12,AUTO,true
            2023-02-05,0.0140292873,true,IPCA,2021-07-01,12,AUTO,true
            2023-03-05,0.0141411588,true,IPCA,2021-07-01,12,AUTO,true
            2023-04-05,0.0142539224,true,IPCA,2021-07-01,12,AUTO,true
            2023-05-05,0.0143675852,true,IPCA,2021-07-01,12,AUTO,true
            2023-06-05,0.0144821543,true,IPCA,2021-07-01,12,AUTO,true
            2023-07-05,0.0145976371,true,IPCA,2021-07-01,12,AUTO,true
            2023-08-05,0.0147140407,true,IPCA,2021-07-01,12,AUTO,true
            2023-09-05,0.0148313725,true,IPCA,2021-07-01,24,AUTO,true
            2023-10-05,0.0149496400,true,IPCA,2021-07-01,24,AUTO,true
            2023-11-05,0.0150688505,true,IPCA,2021-07-01,24,AUTO,true
            2023-12-05,0.0151890116,true,IPCA,2021-07-01,24,AUTO,true
            2024-01-05,0.0153101309,true,IPCA,2021-07-01,24,AUTO,true
            2024-02-05,0.0154322161,true,IPCA,2021-07-01,24,AUTO,true
            2024-03-05,0.0155552747,true,IPCA,2021-07-01,24,AUTO,true
            2024-04-05,0.0156793147,true,IPCA,2021-07-01,24,AUTO,true
            2024-05-05,0.0158043437,true,IPCA,2021-07-01,24,AUTO,true
            2024-06-05,0.0159303698,true,IPCA,2021-07-01,24,AUTO,true
            2024-07-05,0.0160574008,true,IPCA,2021-07-01,24,AUTO,true
            2024-08-05,0.0161854448,true,IPCA,2021-07-01,24,AUTO,true
            2024-09-05,0.0163145098,true,IPCA,2021-07-01,36,AUTO,true
            2024-10-05,0.0164446040,true,IPCA,2021-07-01,36,AUTO,true
            2024-11-05,0.0165757355,true,IPCA,2021-07-01,36,AUTO,true
            2024-12-05,0.0167079128,true,IPCA,2021-07-01,36,AUTO,true
            2025-01-05,0.0168411440,true,IPCA,2021-07-01,36,AUTO,true
            2025-02-05,0.0169754377,true,IPCA,2021-07-01,36,AUTO,true
            2025-03-05,0.0171108022,true,IPCA,2021-07-01,36,AUTO,true
            2025-04-05,0.0172472461,true,IPCA,2021-07-01,36,AUTO,true
            2025-05-05,0.0173847781,true,IPCA,2021-07-01,36,AUTO,true
            2025-06-05,0.0175234068,true,IPCA,2021-07-01,36,AUTO,true
            2025-07-05,0.0176631409,true,IPCA,2021-07-01,36,AUTO,true
            2025-08-05,0.0178039892,true,IPCA,2021-07-01,36,AUTO,true
            2025-09-05,0.0179459608,true,IPCA,2021-07-01,48,AUTO,true
            2025-10-05,0.0180890644,true,IPCA,2021-07-01,48,AUTO,true
            2025-11-05,0.0182333091,true,IPCA,2021-07-01,48,AUTO,true
            2025-12-05,0.0183787041,true,IPCA,2021-07-01,48,AUTO,true
            2026-01-05,0.0185252584,true,IPCA,2021-07-01,48,AUTO,true
            2026-02-05,0.0186729814,true,IPCA,2021-07-01,48,AUTO,true
            2026-03-05,0.0188218824,true,IPCA,2021-07-01,48,AUTO,true
            2026-04-05,0.0189719708,true,IPCA,2021-07-01,48,AUTO,true
            2026-05-05,0.0191232559,true,IPCA,2021-07-01,48,AUTO,true
            2026-06-05,0.0192757474,true,IPCA,2021-07-01,48,AUTO,true
            2026-07-05,0.0194294550,true,IPCA,2021-07-01,48,AUTO,true
            2026-08-05,0.0195843882,true,IPCA,2021-07-01,48,AUTO,true
            2026-09-05,0.0197405568,true,IPCA,2021-07-01,60,AUTO,true
            2026-10-05,0.0198979708,true,IPCA,2021-07-01,60,AUTO,true
            2026-11-05,0.0200566400,true,IPCA,2021-07-01,60,AUTO,true
            2026-12-05,0.0202165745,true,IPCA,2021-07-01,60,AUTO,true
            2027-01-05,0.0203777843,true,IPCA,2021-07-01,60,AUTO,true
            2027-02-05,0.0205402796,true,IPCA,2021-07-01,60,AUTO,true
            2027-03-05,0.0207040707,true,IPCA,2021-07-01,60,AUTO,true
            2027-04-05,0.0208691707,true,IPCA,2021-07-01,60,AUTO,true
        '''.split()))
    )),

    'Ágata - CDI - Livre': list(fincore.build(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('6.33'),
        vir=fincore.VariableIndex('CDI'),
        amortizations=list(map(_AMORT, '''
            2021-12-28,0,false
            2022-01-28,0.05555555555,true
            2022-02-28,0.05555555555,true
            2022-03-28,0.05555555555,true
            2022-04-28,0.05555555555,true
            2022-05-28,0.05555555555,true
            2022-06-28,0.05555555555,true
            2022-07-28,0.05555555555,true
            2022-08-28,0.05555555555,true
            2022-09-28,0.05555555555,true
            2022-10-28,0.05555555555,true
            2022-11-28,0.05555555555,true
            2022-12-28,0.05555555555,true
            2023-01-28,0.05555555555,true
            2023-02-28,0.05555555555,true
            2023-03-28,0.05555555555,true
            2023-04-28,0.05555555555,true
            2023-05-28,0.05555555555,true
            2023-06-28,0.05555555555,true
        '''.split()))
    )),

    'Bossanova CCB5 - Poupança - Bullet': list(fincore.build_bullet(
        principal=decimal.Decimal('1235000'),
        apy=decimal.Decimal('0'),
        zero_date=datetime.date(2022, 6, 23),
        term=120,
        vir=fincore.VariableIndex('Poupança', 118)
    )),

    'Bossanova CCB7 - IPCA - Bullet': list(fincore.build_bullet(
        principal=decimal.Decimal('176000'),
        apy=decimal.Decimal('0'),
        zero_date=datetime.date(2022, 10, 24),
        term=120,
        vir=fincore.VariableIndex('IPCA')
    )),

    'Unique Tower PCI - IPCA - Livre': list(fincore.build(  # PCI = Proteção Contra Inflação.
        principal=decimal.Decimal('1100500'),
        apy=decimal.Decimal('11'),
        vir=fincore.VariableIndex('IPCA'),
        amortizations=list(map(_PLA_AMORT, '''
            2022-07-08,0,false
            2022-08-08,0.033333333333333335,true,IPCA,2022-07-01,01,M-1,true
            2022-09-08,0.033333333333333335,true,IPCA,2022-07-01,02,M-1,true
            2022-10-08,0.033333333333333335,true,IPCA,2022-07-01,03,M-1,true
            2022-11-08,0.033333333333333335,true,IPCA,2022-07-01,04,M-1,true
            2022-12-08,0.033333333333333335,true,IPCA,2022-07-01,05,M-1,true
            2023-01-08,0.033333333333333335,true,IPCA,2022-07-01,06,M-1,true
            2023-02-08,0.033333333333333335,true,IPCA,2022-07-01,07,M-1,true
            2023-03-08,0.033333333333333335,true,IPCA,2022-07-01,08,M-1,true
            2023-04-08,0.033333333333333335,true,IPCA,2022-07-01,09,M-1,true
            2023-05-08,0.033333333333333335,true,IPCA,2022-07-01,10,M-1,true
            2023-06-08,0.033333333333333335,true,IPCA,2022-07-01,11,M-1,true
            2023-08-08,0.033333333333333335,true,IPCA,2022-07-01,12,M-1,true
            2023-09-08,0.033333333333333335,true,IPCA,2022-07-01,13,M-1,true
            2023-10-08,0.033333333333333335,true,IPCA,2022-07-01,14,M-1,true
            2023-11-08,0.033333333333333335,true,IPCA,2022-07-01,15,M-1,true
            2023-12-08,0.033333333333333335,true,IPCA,2022-07-01,16,M-1,true
            2024-01-08,0.033333333333333335,true,IPCA,2022-07-01,17,M-1,true
            2024-02-08,0.033333333333333335,true,IPCA,2022-07-01,18,M-1,true
            2024-03-08,0.033333333333333335,true,IPCA,2022-07-01,19,M-1,true
            2024-04-08,0.033333333333333335,true,IPCA,2022-07-01,20,M-1,true
            2024-05-08,0.033333333333333335,true,IPCA,2022-07-01,21,M-1,true
            2024-06-08,0.033333333333333335,true,IPCA,2022-07-01,22,M-1,true
            2024-07-08,0.033333333333333335,true,IPCA,2022-07-01,23,M-1,true
            2024-08-08,0.033333333333333335,true,IPCA,2022-07-01,24,M-1,true
            2024-09-08,0.033333333333333335,true,IPCA,2022-07-01,25,M-1,true
            2024-10-08,0.033333333333333335,true,IPCA,2022-07-01,26,M-1,true
            2024-11-08,0.033333333333333335,true,IPCA,2022-07-01,27,M-1,true
            2024-12-08,0.033333333333333335,true,IPCA,2022-07-01,28,M-1,true
            2025-01-08,0.033333333333333335,true,IPCA,2022-07-01,29,M-1,true
            2025-02-08,0.033333333333333335,true,IPCA,2022-07-01,30,M-1,true
        '''.split()))
    )),

    'VIC Park Montana e Villa VIC Salerno & Parcelas Amortizadas - CDI - Livre': list(fincore.build(
        principal=decimal.Decimal('2000000'),
        apy=decimal.Decimal('6'),
        vir=fincore.VariableIndex('CDI'),
        amortizations=list(map(_AMORT, '''
            2022-07-15,0,false
            2022-08-15,0.02777777777778,true
            2022-09-15,0.02777777777778,true
            2022-10-15,0.02777777777778,true
            2022-11-15,0.02777777777778,true
            2022-12-15,0.02777777777778,true
            2023-01-15,0.02777777777778,true
            2023-02-15,0.02777777777778,true
            2023-03-15,0.02777777777778,true
            2023-04-15,0.02777777777778,true
            2023-05-15,0.02777777777778,true
            2023-06-15,0.02777777777778,true
            2023-07-15,0.02777777777778,true
            2023-08-15,0.02777777777778,true
            2023-09-15,0.02777777777778,true
            2023-10-15,0.02777777777778,true
            2023-11-15,0.02777777777778,true
            2023-12-15,0.02777777777778,true
            2024-01-15,0.02777777777778,true
            2024-02-15,0.02777777777778,true
            2024-03-15,0.02777777777778,true
            2024-04-15,0.02777777777778,true
            2024-05-15,0.02777777777778,true
            2024-06-15,0.02777777777778,true
            2024-07-15,0.02777777777778,true
            2024-08-15,0.02777777777778,true
            2024-09-15,0.02777777777778,true
            2024-10-15,0.02777777777778,true
            2024-11-15,0.02777777777778,true
            2024-12-15,0.02777777777778,true
            2025-01-15,0.02777777777778,true
            2025-02-15,0.02777777777778,true
            2025-03-15,0.02777777777778,true
            2025-04-15,0.02777777777778,true
            2025-05-15,0.02777777777778,true
            2025-06-15,0.02777777777778,true
            2025-07-15,0.02777777777778,true
        '''.split()))
    )),

    'Crédito p/ Estudantes de Medicina (Mesada Médicos) - Pré-Fixada - Livre': list(fincore.build(
        principal=decimal.Decimal('2000000'),
        apy=decimal.Decimal('21.5'),
        amortizations=list(map(_AMORT, '''
            2022-08-08,0,false
            2022-09-08,0,false
            2022-10-08,0,false
            2022-11-08,0,false
            2022-12-08,0,false
            2023-01-08,0,false
            2023-02-08,0,false
            2023-03-08,0.02608593235,true
            2023-04-08,0.02651272620,true
            2023-05-08,0.02694650285,true
            2023-06-08,0.02738737656,true
            2023-07-08,0.02783546343,true
            2023-08-08,0.02829088149,true
            2023-09-08,0.02875375067,true
            2023-10-08,0.02922419289,true
            2023-11-08,0.02970233205,true
            2023-12-08,0.03018829408,true
            2024-01-08,0.03068220697,true
            2024-02-08,0.03118420081,true
            2024-03-08,0.03169440780,true
            2024-04-08,0.03221296233,true
            2024-05-08,0.03274000096,true
            2024-06-08,0.03327566252,true
            2024-07-08,0.03382008807,true
            2024-08-08,0.03437342101,true
            2024-09-08,0.03493580706,true
            2024-10-08,0.03550739436,true
            2024-11-08,0.03608833344,true
            2024-12-08,0.03667877731,true
            2025-01-08,0.03727888147,true
            2025-02-08,0.03788880398,true
            2025-03-08,0.03850870548,true
            2025-04-08,0.03913874923,true
            2025-05-08,0.03977910117,true
            2025-06-08,0.04042992996,true
            2025-07-08,0.04109140700,true
            2025-08-08,0.04176370650,true
        '''.split()))
    )),

    'Villa das Paineiras - CDI - Livre': list(fincore.build(
        principal=decimal.Decimal('3000000'),
        apy=decimal.Decimal('6.5'),
        vir=fincore.VariableIndex('CDI'),
        amortizations=list(map(_AMORT, '''
            2022-09-19,0,false
            2022-10-19,0,false
            2022-11-19,0,false
            2022-12-19,0,false
            2023-01-19,0,false
            2023-02-19,0,false
            2023-03-19,0,false
            2023-04-19,0.05555555555,true
            2023-05-19,0.05555555555,true
            2023-06-19,0.05555555555,true
            2023-07-19,0.05555555555,true
            2023-08-19,0.05555555555,true
            2023-09-19,0.05555555555,true
            2023-10-19,0.05555555555,true
            2023-11-19,0.05555555555,true
            2023-12-19,0.05555555555,true
            2024-01-19,0.05555555555,true
            2024-02-19,0.05555555555,true
            2024-03-19,0.05555555555,true
            2024-04-19,0.05555555555,true
            2024-05-19,0.05555555555,true
            2024-06-19,0.05555555555,true
            2024-07-19,0.05555555555,true
            2024-08-19,0.05555555555,true
            2024-09-19,0.05555555555,true
        '''.split()))
    )),

    'Residencial da Mata (15%) - Pré-Fixada - Bullet': list(fincore.build_bullet(
        principal=decimal.Decimal('110500.00'),
        apy=decimal.Decimal('15.0'),
        zero_date=datetime.date(2021, 6, 4),
        term=18,
        insertions=[
            fincore.Amortization.Bare(date=datetime.date(2022, 10, 6), value=fincore.Amortization.Bare.MAX_VALUE)
        ],
        capitalisation='365'
    )),

    'Residencial da Mata (15%) - Pré-Fixada - Price': list(fincore.build_price(
        principal=decimal.Decimal('115000.00'),
        apy=decimal.Decimal('15.0'),
        zero_date=datetime.date(2021, 6, 4),
        term=18,
        insertions=[
            fincore.Amortization.Bare(date=datetime.date(2022, 10, 6), value=decimal.Decimal('10000'))
        ]
    )),

    'Residencial da Mata (15.5%) - Pré-Fixada - Bullet': list(fincore.build_bullet(
        principal=decimal.Decimal('634000.00'),
        apy=decimal.Decimal('15.5'),
        zero_date=datetime.date(2021, 6, 4),
        term=18,
        insertions=[
            fincore.Amortization.Bare(date=datetime.date(2022, 10, 6), value=fincore.Amortization.Bare.MAX_VALUE)
        ],
        capitalisation='365'
    )),

    'Residencial da Mata (15.5%) - Pré-Fixada - Price': list(fincore.build_price(
        principal=decimal.Decimal('633500.00'),
        apy=decimal.Decimal('15.5'),
        zero_date=datetime.date(2021, 6, 4),
        term=18,
        insertions=[
            fincore.Amortization.Bare(date=datetime.date(2022, 10, 6), value=decimal.Decimal('50000'))
        ]
    ))
}

XOPS = {
    'Teste 01 (aniversário) - Pré-Fixada - Juros mensais': list(fincore.build_jm(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('20.0'),
        zero_date=datetime.date(2020, 12, 1),
        anniversary_date=datetime.date(2020, 12, 15),
        term=12
    )),

    'Teste 02 (aniversário) - CDI - Livre': list(fincore.build(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('15'),
        vir=fincore.VariableIndex('CDI'),
        amortizations=list(map(_AMORT, '''
            2021-06-06,0,false
            2021-07-01,0.05555555555,true
            2021-08-01,0.05555555555,true
            2021-09-01,0.05555555555,true
            2021-10-01,0.05555555555,true
            2021-11-01,0.05555555555,true
            2021-12-01,0.05555555555,true
            2022-01-01,0.05555555555,true
            2022-02-01,0.05555555555,true
            2022-03-01,0.05555555555,true
            2022-04-01,0.05555555555,true
            2022-05-01,0.05555555555,true
            2022-06-01,0.05555555555,true
            2022-07-01,0.05555555555,true
            2022-08-01,0.05555555555,true
            2022-09-01,0.05555555555,true
            2022-10-01,0.05555555555,true
            2022-11-01,0.05555555555,true
            2022-12-01,0.05555555555,true
        '''.split()))
    )),

    'Teste 03 (adiantamento PARCIAL) - Pré-Fixada - Bullet': list(fincore.build_bullet(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('18.5'),
        zero_date=datetime.date(2022, 2, 10),
        term=12,
        insertions=[
            fincore.Amortization.Bare(date=datetime.date(2022, 5, 21), value=decimal.Decimal(500000)),
            fincore.Amortization.Bare(date=datetime.date(2022, 8, 12), value=decimal.Decimal(100000))
        ]
    )),

    'Teste 04 (adiantamento PARCIAL) - CDI - Bullet': list(fincore.build_bullet(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('10.0'),
        zero_date=datetime.date(2022, 2, 10),
        term=12,
        vir=fincore.VariableIndex('CDI'),
        insertions=[
            fincore.Amortization.Bare(date=datetime.date(2022, 5, 21), value=decimal.Decimal(500000)),
            fincore.Amortization.Bare(date=datetime.date(2022, 8, 12), value=decimal.Decimal(250000))
        ]
    )),

    'Teste 05 (adiantamento PARCIAL) - Pré-Fixada - Juros mensais': list(fincore.build_jm(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('18.5'),
        zero_date=datetime.date(2022, 2, 10),
        term=12,
        insertions=[
            fincore.Amortization.Bare(date=datetime.date(2022, 5, 21), value=decimal.Decimal(500000)),
            fincore.Amortization.Bare(date=datetime.date(2022, 8, 12), value=decimal.Decimal(250000))
        ]
    )),

    'Teste 06 (adiantamento PARCIAL) - CDI - Juros mensais': list(fincore.build_jm(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('10.0'),
        zero_date=datetime.date(2022, 2, 10),
        term=12,
        vir=fincore.VariableIndex('CDI'),
        insertions=[
            fincore.Amortization.Bare(date=datetime.date(2022, 5, 21), value=decimal.Decimal(500000)),
            fincore.Amortization.Bare(date=datetime.date(2022, 8, 12), value=decimal.Decimal(250000))
        ]
    )),

    'Teste 07 (adiantamento PARCIAL) - IPCA - Juros mensais': list(fincore.build_jm(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('10.0'),
        zero_date=datetime.date(2022, 2, 10),
        term=12,
        vir=fincore.VariableIndex('IPCA'),
        insertions=[
            fincore.Amortization.Bare(date=datetime.date(2022, 5, 21), value=decimal.Decimal(500000)),
            fincore.Amortization.Bare(date=datetime.date(2022, 8, 12), value=decimal.Decimal(250000))
        ]
    )),

    'Teste 08 (adiantamento PARCIAL) - Pré-Fixada - Price': list(fincore.build_price(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('18.5'),
        zero_date=datetime.date(2022, 2, 10),
        term=12,
        insertions=[
            fincore.Amortization.Bare(date=datetime.date(2022, 5, 21), value=decimal.Decimal(100000)),
            fincore.Amortization.Bare(date=datetime.date(2022, 8, 12), value=decimal.Decimal(100000))
        ]
    )),

    'Teste 09 (adiantamento PARCIAL) - Pré-Fixada - Livre': list(fincore.build(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('15.5'),
        amortizations=list(map(_AMORT, '''
            2022-02-10,0,false
            2022-03-10,0.05555555555,true
            2022-04-10,0.05555555555,true
            2022-05-10,0.05555555555,true
            2022-06-10,0.05555555555,true
            2022-07-10,0.05555555555,true
            2022-08-10,0.05555555555,true
            2022-09-10,0.05555555555,true
            2022-10-10,0.05555555555,true
            2022-11-10,0.05555555555,true
            2022-12-10,0.05555555555,true
            2023-01-10,0.05555555555,true
            2023-02-10,0.05555555555,true
            2023-03-10,0.05555555555,true
            2023-04-10,0.05555555555,true
            2023-05-10,0.05555555555,true
            2023-06-10,0.05555555555,true
            2023-07-10,0.05555555555,true
            2023-08-10,0.05555555555,true
        '''.split())),
        insertions=[
            fincore.Amortization.Bare(date=datetime.date(2022, 5, 21), value=decimal.Decimal(100000)),
            fincore.Amortization.Bare(date=datetime.date(2022, 8, 12), value=decimal.Decimal(100000)),
            fincore.Amortization.Bare(date=datetime.date(2022, 11, 15), value=decimal.Decimal(100000)),
            fincore.Amortization.Bare(date=datetime.date(2022, 11, 25), value=decimal.Decimal(100000))
        ]
    )),

    'Teste 10 (adiantamento PARCIAL + aniversário) - Pré-Fixada - Livre': list(fincore.build(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('15.5'),
        amortizations=list(map(_AMORT, '''
            2022-02-10,0,false
            2022-03-05,0.08333333333,true
            2022-04-05,0.08333333333,true
            2022-05-05,0.08333333333,true
            2022-06-05,0.08333333333,true
            2022-07-05,0.08333333333,true
            2022-08-05,0.08333333333,true
            2022-09-05,0.08333333333,true
            2022-10-05,0.08333333333,true
            2022-11-05,0.08333333333,true
            2022-12-05,0.08333333333,true
            2023-01-05,0.08333333333,true
            2023-02-05,0.08333333333,true
        '''.split())),
        insertions=[
            fincore.Amortization.Bare(date=datetime.date(2022, 2, 21), value=decimal.Decimal(100000)),
            fincore.Amortization.Bare(date=datetime.date(2022, 8, 12), value=decimal.Decimal(100000)),
            fincore.Amortization.Bare(date=datetime.date(2022, 11, 15), value=decimal.Decimal(100000))
        ]
    )),

    'Teste 11 (adiantamento TOTAL) - Pré-Fixada - Bullet': list(fincore.build_bullet(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('18.5'),
        zero_date=datetime.date(2022, 2, 10),
        term=12,
        insertions=[
            fincore.Amortization.Bare(date=datetime.date(2022, 6, 30), value=fincore.Amortization.Bare.MAX_VALUE)
        ]
    )),

    'Teste 12 (adiantamento TOTAL) - Pré-Fixada - Juros mensais': list(fincore.build_jm(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('18.5'),
        zero_date=datetime.date(2022, 2, 10),
        term=12,
        insertions=[
            fincore.Amortization.Bare(date=datetime.date(2022, 6, 30), value=fincore.Amortization.Bare.MAX_VALUE)
        ]
    )),

    'Teste 13 (adiantamento TOTAL) - Pré-Fixada - Price': list(fincore.build_price(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('18.5'),
        zero_date=datetime.date(2022, 2, 10),
        term=12,
        insertions=[
            # Se caracteriza pela antecipação total da operação por amortizar o percentual restante do principal na data especificada.
            fincore.Amortization.Bare(date=datetime.date(2022, 6, 30), value=fincore.Amortization.Bare.MAX_VALUE)
        ]
    )),

    'Teste 14 (adiantamento TOTAL) - Pré-Fixada - Livre': list(fincore.build(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('18.5'),
        amortizations=list(map(_AMORT, '''
            2022-02-10,0,false
            2022-03-10,0.08333333333,true
            2022-04-10,0.08333333333,true
            2022-05-10,0.08333333333,true
            2022-06-10,0.08333333333,true
            2022-07-10,0.08333333333,true
            2022-08-10,0.08333333333,true
            2022-09-10,0.08333333333,true
            2022-10-10,0.08333333333,true
            2022-11-10,0.08333333333,true
            2022-12-10,0.08333333333,true
            2023-01-10,0.08333333333,true
            2023-02-10,0.08333333333,true
        '''.split())),
        insertions=[
            fincore.Amortization.Bare(date=datetime.date(2022, 6, 30), value=decimal.Decimal(660000))
        ]
    )),

    'Teste 15 (adiantamento PARCIAL + TOTAL) - Pré-Fixada - Bullet': list(fincore.build_bullet(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('21'),
        zero_date=datetime.date(2022, 8, 1),
        term=12,
        insertions=[
            fincore.Amortization.Bare(date=datetime.date(2022, 12, 1), value=decimal.Decimal(200000)),
            fincore.Amortization.Bare(date=datetime.date(2023, 2, 1), value=decimal.Decimal(300000)),
            fincore.Amortization.Bare(date=datetime.date(2023, 6, 1), value=decimal.Decimal(500000))
        ]
    )),

    'Teste 16 (adiantamento PARCIAL + TOTAL) - IPCA - Bullet': list(fincore.build_bullet(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('11'),
        zero_date=datetime.date(2021, 1, 1),
        term=12,
        vir=fincore.VariableIndex('IPCA'),
        insertions=[
            fincore.Amortization.Bare(date=datetime.date(2021, 4, 1), value=decimal.Decimal(200000)),
            fincore.Amortization.Bare(date=datetime.date(2021, 6, 1), value=decimal.Decimal(200000)),
            fincore.Amortization.Bare(date=datetime.date(2021, 8, 1), value=decimal.Decimal(200000))
        ]
    )),

    'Teste 17 (adiantamento PARCIAL + TOTAL) - IPCA - Livre': list(fincore.build(
        principal=decimal.Decimal('1000000'),
        apy=decimal.Decimal('10'),
        amortizations=list(map(_PLA_AMORT, '''
            2022-09-16,0,false
            2022-10-16,0.08333333333,true,IPCA,2021-09-01,01,AUTO,true
            2022-11-16,0.08333333333,true,IPCA,2021-09-01,02,AUTO,true
            2022-12-16,0.08333333334,true,IPCA,2021-09-01,03,AUTO,true
            2023-01-16,0.08333333333,true,IPCA,2021-09-01,04,AUTO,true
            2023-02-16,0.08333333333,true,IPCA,2021-09-01,05,AUTO,true
            2023-03-16,0.08333333334,true,IPCA,2021-09-01,06,AUTO,true
            2023-04-16,0.08333333333,true,IPCA,2021-09-01,06,AUTO,true
            2023-05-16,0.08333333333,true,IPCA,2021-09-01,08,AUTO,true
            2023-06-16,0.08333333334,true,IPCA,2021-09-01,09,AUTO,true
            2023-07-16,0.08333333333,true,IPCA,2021-09-01,10,AUTO,true
            2023-08-16,0.08333333333,true,IPCA,2021-09-01,11,AUTO,true
            2023-09-16,0.08333333334,true,IPCA,2021-09-01,12,AUTO,true
        '''.split())),
        insertions=[
            fincore.Amortization.Bare(date=datetime.date(2023, 5, 1), value=decimal.Decimal(100000)),
            fincore.Amortization.Bare(date=datetime.date(2023, 6, 1), value=decimal.Decimal(100000)),
            fincore.Amortization.Bare(date=datetime.date(2023, 7, 1), value=fincore.Amortization.Bare.MAX_VALUE)
        ]
    )),

    'Teste 18 (adiantamento PARCIAL + carência) - CDI - Livre': list(fincore.build(
        principal=decimal.Decimal('3000000'),
        apy=decimal.Decimal('6.5'),
        vir=fincore.VariableIndex('CDI'),
        amortizations=list(map(_PLA_AMORT, '''
            2022-09-15,0,false
            2022-10-15,0,false
            2022-11-15,0,false
            2022-12-15,0,false
            2023-01-15,0,false
            2023-02-15,0,false
            2023-03-15,0,false
            2023-04-15,0.05555555555,true
            2023-05-15,0.05555555555,true
            2023-06-15,0.05555555555,true
            2023-07-15,0.05555555555,true
            2023-08-15,0.05555555555,true
            2023-09-15,0.05555555555,true
            2023-10-15,0.05555555555,true
            2023-11-15,0.05555555555,true
            2023-12-15,0.05555555555,true
            2024-01-15,0.05555555555,true
            2024-02-15,0.05555555555,true
            2024-03-15,0.05555555555,true
            2024-04-15,0.05555555555,true
            2024-05-15,0.05555555555,true
            2024-06-15,0.05555555555,true
            2024-07-15,0.05555555555,true
            2024-08-15,0.05555555555,true
            2024-09-15,0.05555555555,true
        '''.split())),
        insertions=[
            fincore.Amortization.Bare(date=datetime.date(2023, 5, 5), value=decimal.Decimal(1000000))
        ]
    ))
}


In [79]:
# Python.
import random
import decimal
import datetime
import collections

# Libs.
import pandas
import ipywidgets

# Lista padrão de operações. Pode assumir os valores "OPS" e "XOPS".
_OPS_DICT = OPS

# Valor padrão no componente de operações.
_DEFAULT_OP = random.choice(list(_OPS_DICT.keys()))

def show_schedule(op):
    buf = collections.defaultdict(list)

    for x in _OPS_DICT[op]:
        buf['nº'].append(x.no)
        buf['date'].append(x.date)
        buf['raw'].append(x.raw)
        buf['tax'].append(x.tax)
        buf['net'].append(x.net)
        buf['gain'].append(x.gain)
        buf['amort'].append(x.amort)

        if hasattr(x, 'pla'):
            buf['pla'].append(x.pla)

        else:
            buf['pla'].append(decimal.Decimal())

        buf['bal'].append(x.bal)

    df = pandas.DataFrame(buf).set_index('nº')

    df['date'] = pandas.to_datetime(df['date'])

    return df

def update_display(op):
    output.clear_output(wait=True)

    with output:
        display(show_schedule(op))

select = ipywidgets.Dropdown(options=list(_OPS_DICT.keys()), description='Operação', value=_DEFAULT_OP)
output = ipywidgets.Output()

ipywidgets.interactive(update_display, op=select)

display(ipywidgets.VBox([select, output]))

update_display(_DEFAULT_OP)

# Notas

Para garantir a qualidade do código instale as seguintes bibliotecas.

    python -m pip install nbqa flake8 mypy

Para validar padrão de codificação, rodar o Flake8.

    nbqa flake8 support/lab-000.ipynb

Para validar tipos de dados.

    nbqa mypy support/lab-000.ipynb --ignore-missing-imports --strict

<br/>

---

Rafael Viotti &lt;viotti@inco.vc&gt; e Gustavo Zille &lt;gustavo.zille@inco.vc&gt;<br/>
Março 2022